## Train_Generator

In [1]:
import os;
import sys;
import numpy as np;
import random;
from tensorflow import keras;
import glob;
import math;
import time;

2023-12-28 15:42:10.064272: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-28 15:42:10.091672: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-28 15:42:10.091695: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-28 15:42:10.092366: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-28 15:42:10.096579: I tensorflow/core/platform/cpu_feature_guar

In [2]:
sys.path.append(os.getcwd());
sys.path.append(os.path.join(os.getcwd(), 'common'));

In [3]:
import common.utils as U;
import common.tlopts as tlopts
import resources.TLModels as tlmodels;
import argparse

In [4]:
import torch

In [5]:
np.eye(6)

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])

In [6]:
class TLGenerator(keras.utils.Sequence):
    #Generates data for Keras
    def __init__(self, samples, labels, options):
        random.seed(42);
        #Initialization
        print(f"length of samples:{len(samples)}")
        self.data = [(samples[i], labels[i]) for i in range (0, len(samples))];
        self.opt = options;
        self.batch_size = options.batchSize;
        self.preprocess_funcs = self.preprocess_setup();
        self.mapdict = dict([(17,1),(18,2),(24,3),
                             (51,4),(52,5),(53,6)])

    def __len__(self):
        #Denotes the number of batches per epoch
        return int(np.floor(len(self.data) / self.batch_size));
        #return len(self.samples);

    def __getitem__(self, batchIndex):
        #Generate one batch of data
        batchX, batchY = self.generate_batch(batchIndex);
        batchX = np.expand_dims(batchX, axis=1);
        batchX = np.expand_dims(batchX, axis=3);
        return batchX, batchY

    def generate_batch(self, batchIndex):
        #Generates data containing batch_size samples
        sounds = [];
        labels = [];
        indexes = None;
        for i in range(self.batch_size):
            # Training phase of BC learning
            # Select two training examples
            while True:
                sound1, label1 = self.data[random.randint(0, len(self.data) - 1)]
                sound2, label2 = self.data[random.randint(0, len(self.data) - 1)]
                if label1 != label2:
                    break
            sound1 = self.preprocess(sound1)
            sound2 = self.preprocess(sound2)

            # Mix two examples
            r = np.array(random.random())
            sound = U.mix(sound1, sound2, r, self.opt.sr).astype(np.float32)
            print(f"sound length after U.mix is {len(sound)}")
            eye = np.eye(self.opt.nClasses)
            idx1 = self.mapdict[label1]- 1
            idx2 = self.mapdict[label2] - 1
            label = (eye[idx1] * r + eye[idx2] * (1 - r)).astype(np.float32)
            # label = (eye[label1] * r + eye[label2] * (1 - r)).astype(np.float32)

            #For stronger augmentation
            sound = U.random_gain(6)(sound).astype(np.float32)
            print(f"sound length after U.random_gain is {len(sound)}")
            sounds.append(sound);
            labels.append(label);

        sounds = np.asarray(sounds);
        labels = np.asarray(labels);
        print(f"labels in generate_batch is:\n{labels}")

        return sounds, labels;

    def preprocess_setup(self):
        funcs = []
        if self.opt.strongAugment:
            funcs += [U.random_scale(1.25)]

        funcs += [U.padding(self.opt.inputLength // 2),
                  U.random_crop(self.opt.inputLength),
                  U.normalize(32768.0)]
        return funcs

    def preprocess(self, sound):
        for f in self.preprocess_funcs:
            sound = f(sound)

        return sound;

In [7]:
# def setup(opt, split):
def getTrainGen(opt=None, split=None):
    # dataset = np.load(os.path.join(opt.data, opt.dataset, 'wav{}.npz'.format(opt.sr // 1000)), allow_pickle=True);
    dataset = np.load("datasets/fold1_dataset.npz", allow_pickle=True);
    train_sounds = []
    train_labels = []
    # for i in range(1, opt.nFolds + 1):
    train_sounds = dataset['fold{}'.format(1)].item()['sounds']
    train_labels = dataset['fold{}'.format(1)].item()['labels']
    trainGen = TLGenerator(train_sounds, train_labels, opt);
    return trainGen
    # print(len(train_sounds))
    # print(len(train_labels))
    
        # if i != split:
    # train_sounds.extend(sounds)
    # train_labels.extend(labels)
    # print("sounds:\n")
    # print(train_sounds);
    # print("\n********************************************\n");
    # print("labels:\n")
    # print(train_labels)
    

In [8]:
# getTrainGen()

## Transfer-Learning Trainer

In [9]:
class TLTrainer:
    def __init__(self, opt=None):
        self.opt = opt;
        # self.opt.trainer = self;
        self.trainGen = getTrainGen(self.opt, self.opt.splits)#train_generator.setup(self.opt, self.opt.split);
        self.pretrainedmodelpath = "./resources/pretrained_models/acdnet20_20khz_fold4.h5"
        
    def Train(self):
        # model = tlmodels.GetAcdnetModel(self.opt.inputLength, 50, self.opt.sr, ch_config = self.opt.model_config);
        model  = tlmodels.GetTLACDNet(pretrained_model_path=self.pretrainedmodelpath, opt=self.opt);
        model.summary();

        loss = 'kullback_leibler_divergence';
        optimizer = keras.optimizers.SGD(learning_rate=self.opt.LR, weight_decay=self.opt.weightDecay, momentum=self.opt.momentum, nesterov=True)

        model.compile(loss=loss, optimizer=optimizer , metrics=['accuracy']);

        # learning schedule callback
        lrate = keras.callbacks.LearningRateScheduler(self.GetLR);
        #mark out the following line to not execu modelcheckpoint
        best_model = keras.callbacks.ModelCheckpoint('tf/trained_models/{}.h5'.format(self.opt.model_name), monitor='val_acc', save_best_only=True, verbose=0);
        custom_evaluator = CustomCallback(self.opt);
        # callbacks_list = [lrate, custom_evaluator, best_model];
        callbacks_list = [lrate, custom_evaluator,best_model];
        print(f"Length of trainGen.data:{len(self.trainGen.data)}");
        print(f"Length of batch size:{self.trainGen.batch_size}")
        print(f"type of trainGen: {type(self.trainGen)}")
        model.fit(self.trainGen, epochs=self.opt.nEpochs, steps_per_epoch=len(self.trainGen.data)//self.trainGen.batch_size, callbacks=callbacks_list, verbose=0);
# 
    def GetLR(self, epoch):
        divide_epoch = np.array([self.opt.nEpochs * i for i in self.opt.schedule]);
        decay = sum(epoch > divide_epoch);
        if epoch <= self.opt.warmup:
            decay = 1;
        return self.opt.LR * np.power(0.1, decay);

In [54]:
class CustomCallback(keras.callbacks.Callback):
    def __init__(self, opt):
        self.opt = opt;
        self.testX = None;
        self.testY = None;
        self.curEpoch = 0;
        self.curLr = opt.LR;
        self.cur_epoch_start_time = time.time();
        self.bestAcc = 0.0;
        self.bestAccEpoch = 0;

    def on_epoch_begin(self, epoch, logs=None):
        self.curEpoch = epoch+1;
        self.curLr = TLTrainer(self.opt).GetLR(epoch+1);
        self.cur_epoch_start_time = time.time();

    def on_epoch_end(self, epoch, logs=None):
        train_time = time.time() - self.cur_epoch_start_time;
        print(f"training time:{train_time}")
        self.load_test_data();
        val_acc, val_loss = self.validate(self.model);
        logs['val_acc'] = val_acc;
        logs['val_loss'] = val_loss;
        if val_acc > self.bestAcc:
            self.bestAcc = val_acc;
            self.bestAccEpoch = epoch + 1;
        epoch_time = time.time() - self.cur_epoch_start_time;
        val_time = epoch_time - train_time;
        
        line = 'SP-{}, Epoch: {}/{} | Time: {} (Train {}  Val {}) | Train: LR {}  Loss {:.2f}  Acc {:.2f}% | Val: Loss {:.2f}  Acc(top1) {:.2f}% | HA {:.2f}@{}\n'.format(
            1, epoch+1, self.opt.nEpochs, U.to_hms(epoch_time), U.to_hms(train_time), U.to_hms(val_time),
            self.curLr, logs['loss'], logs['accuracy'] if 'accuracy' in logs else logs['acc'], val_loss, val_acc, self.bestAcc, self.bestAccEpoch);
        sys.stdout.write(line);
        sys.stdout.flush();

    def load_test_data(self):
        if self.testX is None:
            data = np.load('datasets/fold1_test16000.npz', allow_pickle=True);
            self.testX = data['x'];
            self.testY = data['y'];
    # def load_test_data(self):
    #     if self.testX is None:
    #         data = np.load('datasets/fold1_test16000.npz', allow_pickle=True);
    #         self.testX = data['fold{}'.format(1)].item()['sounds']
    #         self.testY = data['fold{}'.format(1)].item()['labels']

    def validate(self, model):
        y_pred = None;
        y_target = None;
        batch_size = 5#(self.opt.batchSize//self.opt.nCrops)*self.opt.nCrops;
        loopend = math.ceil(len(self.testX) / batch_size)
        print(f"in validate loopend is {loopend}")
        batch_size_ = 5
        for batchIndex in range(loopend):
            x = self.testX[batchIndex*batch_size : (batchIndex+1)*batch_size];
            print(f"type of x : {type(x)}")
            y = self.testY[batchIndex*batch_size : (batchIndex+1)*batch_size];
            # for k in range(len(x)):
            #     print(f"itme{x[k]} length is {len(x[k])}")
            #     if len(x[k]) < 80000:
            #         x[k] = np.pad(x[k],(0,80000-len(x[k])),'constant')
            # print(f"length of Test X is {len(x)}")
            # print(f"length of Test Y is {len(y)}")
            
            for itm in x:
                print(f"itm length is {len(itm)}")
            scores = model.predict(x, batch_size=len(y), verbose=0);
            # scores = model.predict(x, batch_size=batch_size_, verbose=0);
            y_pred = scores if y_pred is None else np.concatenate((y_pred, scores));
            y_target = y if y_target is None else np.concatenate((y_target, y));
            #break;

        acc, loss = self.compute_accuracy(y_pred, y_target);
        return acc, loss;

    #Calculating average prediction (10 crops) and final accuracy
    def compute_accuracy(self, y_pred, y_target):
        #Reshape y_pred to shape it like each sample comtains 10 samples.
        if self.opt.nCrops > 1:
            y_pred = (y_pred.reshape(y_pred.shape[0]//self.opt.nCrops, self.opt.nCrops, y_pred.shape[1])).mean(axis=1);
            y_target = (y_target.reshape(y_target.shape[0]//self.opt.nCrops, self.opt.nCrops, y_target.shape[1])).mean(axis=1);

        loss = keras.losses.KLD(y_target, y_pred).numpy().mean();

        #Get the indices that has highest average value for each sample
        y_pred = y_pred.argmax(axis=1);
        y_target = y_target.argmax(axis=1);
        accuracy = (y_pred==y_target).mean()*100;

        return accuracy, loss;

## getOpts method is for generate options for training

In [55]:
def getOpts():
    parser = argparse.ArgumentParser(description='Transfer Learning for ACDNet');
    parser.add_argument('--netType', default='TLACDNet',  required=False);
    parser.add_argument('--data', default='{}/datasets/processed/'.format(os.getcwd()),  required=False);
    parser.add_argument('--dataset', required=False, default='uec_iot', choices=['10']);
    parser.add_argument('--BC', default=True, action='store_true', help='BC learning');
    parser.add_argument('--strongAugment', default=True,  action='store_true', help='Add scale and gain augmentation');
    #在ipynb中，不能使用parser.parse，要改用parser.parse_known_args()
    opt, unknown = parser.parse_known_args()
    #Leqarning settings
    opt.batchSize = 5;#64;
    opt.weightDecay = 5e-4;
    opt.momentum = 0.09;
    opt.nEpochs = 10;#2000;
    opt.LR = 0.01#0.1;
    opt.schedule = [0.03, 0.06, 0.09]#[0.3, 0.6, 0.9];
    opt.warmup = 10;

    #Basic Net Settings
    opt.nClasses = 6#50;
    opt.nFolds = 1;#5;
    opt.splits = 1#[i for i in range(1, opt.nFolds + 1)];
    opt.sr = 16000#20000;
    opt.inputLength = 30225;
    #Test data
    opt.nCrops = 5;
    return opt
    # opt = parser.parse_args();

In [56]:
def main():
    # opt = tlopts.parse();
    opt = getOpts()
    opt.sr = 16000#20000;
    opt.inputLength = 30225;
    opt.trainer = None
    # import torch;
    opt.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu");
    tlopts.display_info(opt)
    opt.model_name = "acdnet_tl_model"
    # valid_path = False;
    print("Initializing TLTrainer Object.....")
    trainer = TLTrainer(opt);
    print("Start to training.....")
    trainer.Train();

In [57]:
main()

+------------------------------+
| TLACDNet Sound classification
+------------------------------+
| dataset  : uec_iot
| nEpochs  : 10
| LRInit   : 0.01
| schedule : [0.03, 0.06, 0.09]
| warmup   : 10
| batchSize: 5
| nFolds: 1
| Splits: 1
+------------------------------+
Initializing TLTrainer Object.....
length of samples:65
Start to training.....
original model loaded....
new model info:

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 30225, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 1, 15109, 4)       36        
                                                                 
 batch_normalization (Batch  (None, 1, 15109, 4)       16        
 Normalization)                                                  
                                                            

C:\Users\UEC\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


length of samples:65
sound length after U.mix is 30225
sound length after U.random_gain is 30225
sound length after U.mix is 30225
sound length after U.random_gain is 30225
sound length after U.mix is 30225
sound length after U.random_gain is 30225
sound length after U.mix is 30225
sound length after U.random_gain is 30225
sound length after U.mix is 30225
sound length after U.random_gain is 30225
labels in generate_batch is:
[[0.         0.4195198  0.         0.         0.         0.58048016]
 [0.         0.         0.7221286  0.         0.27787134 0.        ]
 [0.         0.         0.39627397 0.         0.603726   0.        ]
 [0.         0.17059533 0.         0.82940465 0.         0.        ]
 [0.9852215  0.         0.         0.         0.         0.01477848]]
sound length after U.mix is 30225
sound length after U.random_gain is 30225
sound length after U.mix is 30225
sound length after U.random_gain is 30225
sound length after U.mix is 30225
sound length after U.random_gain is 30